In [38]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd

In [39]:
user='G'

if user=='S':
    mandat_dep=pd.read_csv("C:/Users/sylva/Desktop/git/Projet-Python-deputes/liste_mandat.csv")#on récupère le csv de la liste des mandats
    df_depu=pd.read_csv("C:/Users/sylva/Desktop/git/Projet-Python-deputes/listeDepute_complete.csv")

if user=='G':
    mandat_dep=pd.read_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/liste_mandatdef.csv") 
    df_depu=pd.read_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/listeDepute_complete.csv")

<h1> I. Base mandat

### Extraction de la date de début du mandat

In [40]:
debut_mandat=[]
for dates in mandat_dep["Mandat"]:
    dates=str(dates).split()
    debut_mandat.append(' '.join(dates[1:4]))
    
mandat_dep["Debut Mandat"]=np.array(debut_mandat)
mandat_dep.head(2)

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Nom,Debut Mandat
0,0,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,Damien Meslot,19 juin 2002
1,0,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,Damien Meslot,20 juin 2007


### Etablissement de l'âge du député lors de son mandat

In [66]:
df_depu.head(2)

,Unnamed: 0,Nom,Né le,Département de naissance,Département de mort
0,0,Ludovic Pajot,18 novembre 1993,NaN,NaN
1,1,Typhanie Degois,6 janvier 1993,NaN,NaN


In [67]:
mandat_dep.head(2)

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Nom,Debut Mandat
0,0,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,Damien Meslot,19 juin 2002
1,0,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,Damien Meslot,20 juin 2007


In [69]:
#De même que dans le fichier Webscrapp, cette ligne va poser problème
age_dep=mandat_dep.set_index('Nom').join(df_depu[["Nom","Né le","Département de naissance"]].set_index('Nom'), on='Nom')
mandat_dep.reset_index()
mois={'janvier':1,'février':2, 'mars':3, 'avril':4, 'mai':5, 'juin':6, 'juillet':7, 'août':8, 'septembre':9, 'octobre':10, 'novembre':11, 'décembre':12}
def transtype_date(x):
    x=str(x)        
    date_texte=x.split() 
    if len(date_texte)>1 :
        if  date_texte[1] in (mois.keys()):
            if date_texte[0]=="1er":
                date_texte[0]='1'
            date_texte[1]=str(mois[date_texte[1]])
            date_nombre='/'.join(date_texte)
            return pd.to_datetime(date_nombre)
    else:
        return pd.NaT


age_dep["Date de naissance"]=age_dep["Né le"].apply(lambda x: transtype_date(x))
age_dep["Date debut mandat"]=age_dep["Debut Mandat"].apply(lambda x: transtype_date(x))
age_dep.head(2)

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Debut Mandat,Né le,Département de naissance,Date de naissance,Date debut mandat
Nom,,,,,,,,,,,
Damien Meslot,0,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,19 juin 2002,11 novembre 1964,Inconnu,1964-11-11,2002-06-19
Damien Meslot,0,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,20 juin 2007,11 novembre 1964,Inconnu,1964-11-11,2007-06-20


In [70]:
age_dep['age depute']=age_dep['Date debut mandat']-age_dep['Date de naissance']#on crée la variable age au debut du mandat, en jour
age_dep['age en année']=age_dep['age depute'].apply(lambda x: float(x.days / 365.25))#et on la transforme en années
#age_dep=age_dep[['Département','Groupe', 'Législature', 'Mandat', 'Régime politique', 'age en année','Date debut mandat']]

In [71]:
age_dep.head(2)

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Debut Mandat,Né le,Département de naissance,Date de naissance,Date debut mandat,age depute,age en année
Nom,,,,,,,,,,,,,
Damien Meslot,0,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,19 juin 2002,11 novembre 1964,Inconnu,1964-11-11,2002-06-19,13734 days,37.601643
Damien Meslot,0,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,20 juin 2007,11 novembre 1964,Inconnu,1964-11-11,2007-06-20,15561 days,42.603696


On remarque que certains âges sont négatifs! C'est toujours à cause des problèmes que l'on a à cause des députés ayant le même nom. Pour de plus amples explications et des exemples, aller voir le fichier 'analyses nan'.

In [72]:
age_dep[age_dep["age en année"]<0].head(2)

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Debut Mandat,Né le,Département de naissance,Date de naissance,Date debut mandat,age depute,age en année
Nom,,,,,,,,,,,,,
Paul Giacobbi,0,Corse,Radical et radical-socialiste,Ire Assemblée nationale constituante,Du 21 octobre 1945 au 10 juin 1946,Gouvernement provisoire de la République franç...,21 octobre 1945,4 juin 1957,Inconnu,1957-04-06,1945-10-21,-4185 days,-11.457906
Paul Giacobbi,0,Corse,Radical et radical-socialiste,Ire Assemblée nationale constituante,Du 21 octobre 1945 au 10 juin 1946,Gouvernement provisoire de la République franç...,21 octobre 1945,4 juin 1957,Haute-Corse,1957-04-06,1945-10-21,-4185 days,-11.457906


In [73]:
df_depu.groupby("Nom").count().sort_values(by="Né le").tail(5)

,Unnamed: 0,Né le,Département de naissance,Département de mort
Nom,,,,
Jean Rousseau,4,4,4,4
Pierre Dumas,9,9,9,9
Pierre Charles,9,9,9,9
Pierre Leroy-Beaulieu,9,9,9,9
Louis Brunet,9,9,9,9


<h1> II.Table député

<h3> Nombre de mandat par député

In [12]:
nb_mandat=age_dep.groupby("Nom").count()
nb_mandat=pd.DataFrame(nb_mandat).rename(columns={"age en année":"nombre de mandat"})
nb_mandat.head()

,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Debut Mandat,Né le,Date de naissance,Date debut mandat,age depute,nombre de mandat
Nom,,,,,,,,,,,,
"(Louis, Camille, Alexandre, Marie Dubochet",1,1,0,1,1,1,1,1,1,1,1,1
"Aaron, Jean-François Crassous",1,1,1,1,1,1,1,1,1,1,1,1
Abbas Gueye,1,1,1,1,1,1,1,1,1,1,1,1
Abbes Moulessehoul,1,1,1,1,1,1,1,1,1,1,1,1
Abdallah Tebib,1,1,1,1,1,1,1,1,1,1,1,1


In [13]:
df_depu_b=df_depu.join(nb_mandat["nombre de mandat"],on="Nom",lsuffix=" ")
df_depu_b.sample(5)

,Unnamed: 0,Nom,Né le,Département de naissance,Département de mort,nombre de mandat
14643,14643,"Pierre, Bruno Boissier",20 mars 1756,Rhône,Inconnu,2.0
14661,14661,Olivier Provost du Bourion,5 février 1756,Mayenne,Mayenne,2.0
7068,7068,"Charles, Raphaël, Martial Rousse",26 janvier 1860,Var,Inconnu,3.0
5827,5827,Robert Jardel,6 septembre 1880,Vosges,Inconnu,2.0
13850,13850,"Charles, Bazile Mercier-Vergerie",18 janvier 1762,Vendée,Paris,1.0


In [14]:
premier_mandat=age_dep.groupby("Nom")["age en année"].min()
premier_mandat.head()

Nom
(Louis, Camille, Alexandre, Marie Dubochet    45.689254
Aaron, Jean-François Crassous                 49.267625
Abbas Gueye                                   37.470226
Abbes Moulessehoul                            38.811773
Abdallah Tebib                                60.766598
Name: age en année, dtype: float64

In [15]:
age_dep=age_dep.reset_index()

In [16]:
premier_mandat_dpt=age_dep.groupby("Nom")[["Département","Nom"]].head(1)
premier_mandat_dpt=pd.DataFrame(premier_mandat_dpt).rename(columns={"Département":"premier dpt"}).set_index("Nom")
premier_mandat_dpt.head(3)

,premier dpt
Nom,
Damien Meslot,Territoire-de-Belfort
Christine Cloarec-Le Nabour,Ille-et-Vilaine
Pascal Terrasse,Ardèche


<h3> Caractéristiques du premier mandat

In [17]:
df_depu_b=df_depu_b.join(premier_mandat,on="Nom",lsuffix="")
df_depu_b=df_depu_b.join(premier_mandat_dpt,on="Nom",lsuffix="")

In [18]:
df_depu_b.sample(5)

,Unnamed: 0,Nom,Né le,Département de naissance,Département de mort,nombre de mandat,age en année,premier dpt
6520,6520,"Marcel, Gilles Cachin",20 septembre 1869,Côtes,Inconnu,9.0,45.037645,Seine
11250,11250,François Leclerc,30 novembre 1796,Meurthe,Inconnu,1.0,51.392197,Meurthe
4881,4881,"Marcel, Henri Bugain",22 février 1898,Aisne,Aisne,1.0,40.930869,Aisne
16431,16431,Philippe Sermattei,NaN,Inconnu,Inconnu,0.0,NaN,Départements de l'Empire français aujourd'hui...
12416,12416,"Louis, Claude Boucher",9 août 1778,Orne,Inconnu,3.0,42.179329,Orne


In [19]:
df_depu_b["élection locale"]=(" "+df_depu_b["Département de naissance"])==df_depu_b["premier dpt"]

In [20]:
df_depu_b.sample(5)

,Unnamed: 0,Nom,Né le,Département de naissance,Département de mort,nombre de mandat,age en année,premier dpt,élection locale
5560,5560,Pierre Perreau-Pradier,5 juillet 1885,Yonne,Yonne,7.0,26.877481,Yonne,True
4085,4085,Joseph Klock,4 février 1908,Moselle,Bas-Rhin,2.0,43.206023,Bas-Rhin,False
5538,5538,Fernand Mauroux,22 novembre 1885,Gers,Gers,3.0,59.909651,Gers,True
12772,12772,"Pierre, Mathias, Joseph Wartelle d'Herlincourt",31 janvier 1773,Inconnu,Inconnu,3.0,42.275154,Pas-de-Calais,False
2581,2581,Pierre Leroy-Beaulieu,4 août 1798,Seine,Hérault,45.0,-78.759754,Hérault,False


<h1> III. Base département

In [21]:
dep_possible=list(mandat_dep['Département'].unique())#une liste de tous les départements
dep_possible.remove(" Département d'élection inconnu")
dep_possible.remove(np.nan)

In [22]:
dep_possible[:3]

[' Territoire-de-Belfort', ' Ille-et-Vilaine', ' Ardèche']

In [23]:
def pourcentageTerroir(basedep) :#une fonction qui donne le pourcentage de mandats occupés par des députés nés dans leur département d'élection
    basedep_1=basedep[basedep['Département']==' '+basedep['Département de naissance']]
    nbreterroir=basedep_1[basedep_1['Département de naissance']!='Inconnu']['Département'].count()
    i=basedep[basedep['Département de naissance']!='Inconnu']['Département'].count()
    if i!=0:
        pourcTerroir=nbreterroir/i
    else:
        return np.nan
    return pourcTerroir

In [24]:
df=age_dep.reset_index()

In [34]:
dept_naiss=age_dep.set_index('Nom').join(df_depu[["Nom","Né le", "Département de naissance"]].set_index('Nom'), on='Nom', lsuffix=" ")
dept_naiss=dept_naiss.reset_index()
dept_naiss.head()

,Nom,Unnamed: 0,Département,Groupe,Législature,Mandat,Régime politique,Debut Mandat,Né le,Date de naissance,Date debut mandat,age depute,age en année,Né le,Département de naissance
0,Damien Meslot,0,Territoire-de-Belfort,Union pour la majorité présidentielle,XIIe législature,Du 19 juin 2002 au 19 juin 2007,Cinquième République - Assemblée nationale,19 juin 2002,11 novembre 1964,1964-11-11,2002-06-19,13734 days,37.601643,11 novembre 1964,Inconnu
1,Damien Meslot,0,Territoire-de-Belfort,Union pour un mouvement populaire,XIIIe législature,Du 20 juin 2007 au 19 juin 2012,Cinquième République - Assemblée nationale,20 juin 2007,11 novembre 1964,1964-11-11,2007-06-20,15561 days,42.603696,11 novembre 1964,Inconnu
2,Damien Meslot,0,Territoire-de-Belfort,Union pour un mouvement populaire,XIVe législature,Du 20 juin 2012 au 20 juin 2017,Cinquième République - Assemblée nationale,20 juin 2012,11 novembre 1964,1964-11-11,2012-06-20,17388 days,47.605749,11 novembre 1964,Inconnu
3,Christine Cloarec-Le Nabour,0,Ille-et-Vilaine,La République en Marche,XVe législature (Législature en cours),Depuis le 21 juin 2017 (mandat en cours),Cinquième République - Assemblée nationale,le 21 juin,28 octobre 1964,1964-10-28,NaT,NaT,NaN,28 octobre 1964,Inconnu
4,Pascal Terrasse,0,Ardèche,Socialiste,XIe législature,Du 1er juin 1997 au 18 juin 2002,Cinquième République - Assemblée nationale,1er juin 1997,26 octobre 1964,1964-10-26,1997-01-06,11760 days,32.197125,26 octobre 1964,Gard


In [35]:

w,x,y,z,u=[],[],[],[],[]
for dep in dep_possible :
    w+=[df[df['Département']==dep]['age en année'].mean()]#on prend la moyenne d'age dans le département
    x+=[len(df[df['Département']==dep])]#on compte le nombre total de mandat dans le département
    y+=[len(df[df['Département']==dep]['Nom'].unique())]#Puis le nombre de députés différent par dpt
    z+=[x[-1]/y[-1]] #Ceci est le nombre moyen de mandat par député
    u+=[pourcentageTerroir(dept_naiss[dept_naiss["Département"]==dep])]
    
    
carac_dep_bis=np.array([w, x, y, z,u])#on crée un dataframe avec les informations
table_depmt = pd.DataFrame(carac_dep_bis, index=['age moyen des deputes', 'nombre_mandat', 'nombre député','nombre moyen de mandats par député', 'locaux'],columns=dep_possible)
table_depmt.T.sample(10)

,age moyen des deputes,nombre_mandat,nombre député,nombre moyen de mandats par député,locaux
Etablissements français de l’Inde,46.615719,23.0,14.0,1.642857,0.000000
Manche,55.757564,465.0,177.0,2.627119,0.744102
Départements de l'Empire français aujourd'hui en Belgique,44.775862,98.0,83.0,1.180723,0.000000
Vosges,48.636473,355.0,159.0,2.232704,0.670487
Yonne,48.862315,357.0,147.0,2.428571,0.505519
Marne,48.358941,389.0,156.0,2.493590,0.563855
Ardèche,52.584768,331.0,132.0,2.507576,0.503311
Vienne,49.066262,305.0,133.0,2.293233,0.749141
Loire-Atlantique,53.669819,145.0,56.0,2.589286,0.053659
Aisne,49.248617,456.0,190.0,2.400000,0.556936


# Régimes recalculés

In [61]:
import urllib#pour récupérer les données
import bs4#pour rendre lisibles les données
import re
def Scrap (url):
    req = urllib.request.Request(url)
    html = urllib.request.urlopen(req).read()
    page = bs4.BeautifulSoup(html, "lxml")
    return page
def ToDf (table) :
    Listgrossiere=pd.read_html(str(table))#on transforme le tableau en liste de dataframes
    listeDf=Listgrossiere[0]#on récupère le dataframe
    return listeDf
page=Scrap('https://fr.wikipedia.org/wiki/Liste_des_r%C3%A9gimes_fran%C3%A7ais')
table = page.find('table')#on récupère le seul tableau de la page qui liste des députés
listeReg=ToDf(table)
listeReg2=pd.DataFrame(np.array([listeReg[('Nom','Nom')],listeReg[('Date','Date')]]).T,columns=["Nom","Date"])#on s'occupe des doubles index du tableau de wikipedia
lstRegime=listeReg2[16:]#on enlève les lignes correspondants aux régimes trop anciens

In [62]:
debut_regime=[]
import datetime#on importe la date du jour
maintenant=pd.to_datetime(str(datetime.datetime.now().year)+'/'+str(datetime.datetime.now().month)+'/'+str(datetime.datetime.now().day))
def extraire_date_1(x, Debut=True):
    dates=str(x)
    dates=str(dates).split()
    if Debut:
        dates=' '.join(dates[1:4])
    else:
        dates=' '.join(dates[5:])
    return transtype_date(dates)
    
    
lstRegime['Date Debut']=lstRegime['Date'].apply(lambda x : extraire_date_1(x))
lstRegime['Date Fin']=lstRegime['Date'].apply(lambda x : extraire_date_1(x,Debut=False))
lstRegime['Date Fin'][lstRegime['Nom']=='Cinquième République']=maintenant
lstRegime['Durée']=lstRegime['Date Fin']-lstRegime['Date Debut']
lstRegime['Durée en année']=lstRegime['Durée'].apply(lambda x: float(x.days / 365.25))

<ipython-input-62-998a27b9109e>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lstRegime['Date Debut']=lstRegime['Date'].apply(lambda x : extraire_date_1(x))
<ipython-input-62-998a27b9109e>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lstRegime['Date Fin']=lstRegime['Date'].apply(lambda x : extraire_date_1(x,Debut=False))
<ipython-input-62-998a27b9109e>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

In [63]:
lstRegime=lstRegime[~lstRegime["Date Debut"].isna()].drop_duplicates()
lstRegime=lstRegime.reset_index().drop(['index'], axis=1)#on enlève l'ancien index des régimes
lstRegime=lstRegime.set_index('Date Debut')#et on le remplace par sa date du début : il est dans l'ordre chronologique


In [64]:
lstRegime.head()

,Nom,Date,Date Fin,Durée,Durée en année
Date Debut,,,,,
1804-05-18,Premier Empire,du 18 mai 1804 au 4 avril 1814,1814-04-04,3608 days,9.878166
1814-06-04,Première Restauration,du 6 avril 1814 au 20 mars 1815,1815-03-20,289 days,0.791239
1815-03-20,Cent-Jours,du 20 mars 1815 au 7 juillet 1815,1815-07-07,109 days,0.298426
1815-08-07,Seconde Restauration,du 8 juillet 1815 au 2 août 1830,1830-02-08,5299 days,14.507871
1830-09-08,Monarchie de Juillet,du 9 août 1830 au 24 février 1848,1848-02-24,6378 days,17.462012


In [65]:
lstRegime.to_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/regime_wiki.csv")

In [27]:
table_depute=df_depu_b
table_depute.to_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/table_depute.csv")

In [74]:
age_dep.to_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/table_mandat.csv")

In [37]:
table_depmt.to_csv("C:/Users/Administrateur.LAPTOP-FH8TB47B/Desktop/Projet_assemblee/table_dpmt.csv")